## Setting up the Data

In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
# set random seed to ensure that results are repeatable
np.random.seed(1)

## Loading the data

In [2]:
UBank = pd.read_csv("./UniversalBank.csv")

UBank.head(3)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0


In [3]:
UBank.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [4]:
UBank=UBank.drop(["ID"], axis=1)

In [5]:
UBank.head()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [6]:
UBank=pd.get_dummies(UBank, prefix=['Education','Family'], columns=['Education','Family'])
UBank.head()

,Age,Experience,Income,ZIP Code,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3,Family_1,Family_2,Family_3,Family_4
0,25,1,49,91107,1.6,0,0,1,0,0,0,1,0,0,0,0,0,1
1,45,19,34,90089,1.5,0,0,1,0,0,0,1,0,0,0,0,1,0
2,39,15,11,94720,1.0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,35,9,100,94112,2.7,0,0,0,0,0,0,0,1,0,1,0,0,0
4,35,8,45,91330,1.0,0,0,0,0,0,1,0,1,0,0,0,0,1


## Spliting the Data 

In [13]:
X = UBank[['Age', 'Experience', 'Income', 'ZIP Code', 'CCAvg', 'Mortgage',
       'Personal Loan', 'Securities Account', 'Online',
       'CreditCard', 'Education_1', 'Education_2', 'Education_3', 'Family_1',
       'Family_2', 'Family_3', 'Family_4']]
y = UBank[['CD Account']]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [15]:
scaler = StandardScaler()

In [16]:
Array =['Age', 'Experience', 'Income', 'ZIP Code', 'CCAvg', 'Mortgage']

In [17]:
X_train[Array] = scaler.fit_transform(X_train[Array])
X_test[Array] = scaler.transform(X_test[Array])

## Model the data

In [18]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## Logistic Regression Model

In [19]:
log_reg_model = LogisticRegression(max_iter=900)
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [21]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.0,0.7,0.823529
0,default logistic,0.982857,1.0,0.7,0.823529


## L1 Regularization

In [22]:
log_reg_L1_model = LogisticRegression(solver='liblinear', penalty='l1')
_ = log_reg_L1_model.fit(X_train, np.ravel(y_train))

In [23]:
model_preds = log_reg_L1_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"L1 logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.0,0.7,0.823529
0,default logistic,0.982857,1.0,0.7,0.823529
0,L1 logistic,0.982857,1.0,0.7,0.823529


In [38]:
core_measure = "recall"
LR=LogisticRegression()
kfolds = 5
param_grid = {'C': [0.1, 1, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','lasso','elastic']} 
  
grid = RandomizedSearchCV(LR, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END C=0.001, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.938 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.938 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.938 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.937 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.937 total time=   0.0s
[CV 1/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[C

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.975 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END C=10, penalty=l1, solver=liblinear;, score=0.977 total time=   0.5s


...

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END C=10, penalty=l1, solver=liblinear;, score=0.980 total time=   0.5s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END C=10, penalty=l1, solver=liblinear;, score=0.977 total time=   0.5s
[CV 1/5] END C=0.001, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 2/5] END C=10, penalty=l

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [39]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [40]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.975 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.958 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.958 total time=   0.0s
[CV 4/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.966 total time=   0.0s
[CV 5/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.960 total time=   0.0s
[CV 1/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.954 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.945 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.951 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.955 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.946 total time=   0.0s
[CV 1/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=0.1, p

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/

...

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END C=10, penalty=l1, solver=liblinear;, score=0.977 total time=   0.5s
[CV 1/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 2/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.974 total time=   0.0s
[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.975 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.975 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=l

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the f

In [41]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## SVM classification model using Linear kernel

In [24]:
svm_lin_model = SVC(kernel="linear")
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [25]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.0,0.7,0.823529
0,default logistic,0.982857,1.0,0.7,0.823529
0,L1 logistic,0.982857,1.0,0.7,0.823529
0,svm with linear kernel,0.982857,1.0,0.7,0.823529


## SVM classification model using RBF kernel

In [26]:
svm_rbf_model = SVC(kernel="rbf", C=10)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [27]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231


## SVM classification model using polynomial kernel

In [28]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [29]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231
0,svm with polynomial kernel,0.973143,0.804598,0.70,0.748663


In [42]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.974 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.975 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.977 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.958 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.958 total time=   0.0s
[CV 4/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.966 total time=   0.0s
[CV 5/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.960 total time=   0.0s
[CV 1/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.954 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.945 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.951 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.955 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.946 total time=   0.0s
[CV 1/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=0.1, p

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 4/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5]

...

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.977 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.974 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.975 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.977 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .C=10, pen

C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Val

In [44]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [45]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear','poly','rbf']} 
  
grid = RandomizedSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.977 total time=   0.0s
[CV 2/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.974 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.975 total time=   0.0s
[CV 4/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.980 total time=   0.0s
[CV 5/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.977 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END .........C=1, gamma=1, kernel=poly;, score=0.940 total time=   0.3s


...

[CV 5/5] END .....C=10, gamma=0.001, kernel=rbf;, score=0.940 total time=   0.1s
[CV 1/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.977 total time=   0.0s
[CV 2/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.974 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.975 total time=   0.0s
[CV 4/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.980 total time=   0.0s
[CV 5/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.977 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.9766153846153844
... with parameters: {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}


In [46]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.977 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.974 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.975 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.980 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.977 total time=   0.0s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.949 total time=   0.2s


...

C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END ....C=1000, gamma=1, kernel=linear;, score=0.977 total time= 4.2min


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END ....C=1000, gamma=1, kernel=linear;, score=0.974 total time= 4.3min


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Decision Tree Classifier

In [30]:
deci_tree=DecisionTreeClassifier()
deci_tree=deci_tree.fit(X_train, y_train)

In [31]:
c_matrix = confusion_matrix(y_test, deci_tree.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231
0,svm with polynomial kernel,0.973143,0.804598,0.70,0.748663
0,DT,0.966857,0.698113,0.74,0.718447


In [ ]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## Random Search

In [32]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,60), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.648048780487805
... with parameters: {'min_samples_split': 26, 'min_samples_leaf': 7, 'min_impurity_decrease': 0.0011, 'max_leaf_nodes': 92, 'max_depth': 58, 'criterion': 'entropy'}


C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [33]:
c_matrix = confusion_matrix(y_test,rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT_random", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231
0,svm with polynomial kernel,0.973143,0.804598,0.70,0.748663
0,DT,0.966857,0.698113,0.74,0.718447
0,DT_random,0.976000,0.862500,0.69,0.766667


## Grid Search

In [34]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(25,30),  
    'min_samples_leaf': np.arange(1,5),
    'min_impurity_decrease': np.arange(0.0000, 0.0004, 0.0001),
    'max_leaf_nodes': np.arange(187,192), 
    'max_depth': np.arange(29,33), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
The best recall score is 0.6628048780487805
... with parameters: {'criterion': 'entropy', 'max_depth': 29, 'max_leaf_nodes': 187, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 1, 'min_samples_split': 29}


In [35]:
c_matrix = confusion_matrix(y_test,grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT_grid_Search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231
0,svm with polynomial kernel,0.973143,0.804598,0.70,0.748663
0,DT,0.966857,0.698113,0.74,0.718447
0,DT_random,0.976000,0.862500,0.69,0.766667
0,DT_grid_Search,0.972000,0.793103,0.69,0.737968


In [36]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,svm with rbf kernel,0.977714,0.942029,0.65,0.769231
0,DT_random,0.976000,0.862500,0.69,0.766667
0,DT_grid_Search,0.972000,0.793103,0.69,0.737968
0,default logistic,0.982857,1.000000,0.70,0.823529
0,default logistic,0.982857,1.000000,0.70,0.823529
0,L1 logistic,0.982857,1.000000,0.70,0.823529
0,svm with linear kernel,0.982857,1.000000,0.70,0.823529
0,svm with polynomial kernel,0.973143,0.804598,0.70,0.748663
0,DT,0.966857,0.698113,0.74,0.718447


## Recall for the DT is 74%, it is better than both the Logistic Regression and SVM. 